In [ ]:
import json
import pandas as pd
from eventtrader import keys
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.tools import tool


### declare mcp_client once globally

In [2]:
mcp_client = MultiServerMCPClient({
"neo4j": {
    "url": "http://localhost:31380/mcp",
    "transport": "streamable_http",
        }
    })

Neo4j MCP Tool

In [3]:
@tool
async def run_cypher_query(query: str) -> str:
    """Executes a Cypher query on the Neo4j MCP server."""

    tools = await mcp_client.get_tools()
    read_tool = next(t for t in tools if t.name == "read_neo4j_cypher")
    result = await read_tool.ainvoke({"query": query})
    return json.loads(result)



### Find all XBRL CONCEPT that contain the word 'revenue'

In [ ]:
cypher_query = """MATCH (c:Concept)
WHERE toLower(c.qname) CONTAINS toLower('revenue')
RETURN DISTINCT c.qname, c.label"""

data = await run_cypher_query.ainvoke(cypher_query)
df = pd.DataFrame(data)
df.head()

In [ ]:
# Example usage:
cypher_query = "MATCH (n:News) RETURN n LIMIT 1;"
data = await run_cypher_query.ainvoke(cypher_query)
data[0]["n"]

[{'n': {'returns_schedule': '{"hourly": "2023-01-05T15:31:29-05:00", "session": "2023-01-05T16:00:00-05:00", "daily": "2023-01-05T16:00:00-05:00"}',
   'created': '2023-01-05T14:31:29-05:00',
   'title': "Darling Ingredients Says We're Maintaining Guidance For 2022, Giving Guidance Of 20% Growth For 2023",
   'body': '',
   'url': 'https://www.benzinga.com/news/23/01/30300128/darling-ingredients-says-were-maintaining-guidance-for-2022-giving-guidance-of-20-growth-for-2023',
   'tags': '[]',
   'market_session': 'in_market',
   'channels': '["News", "Guidance"]',
   'id': 'bzNews_30300128',
   'embedding': [0.018065182492136955,
    0.010608648881316185,
    -0.014585962519049644,
    0.014429843984544277,
    0.04960119351744652,
    -0.006070050410926342,
    -0.037884846329689026,
    0.011664309538900852,
    -0.03488141670823097,
    0.0017442565876990557,
    0.026302311569452286,
    0.013225498609244823,
    0.03045061230659485,
    -0.023165065795183182,
    -0.0212619025260210

In [5]:
cypher_query = """MATCH (c:Company)<-[:PRIMARY_FILER]-(r:Report)-[:HAS_SECTION]->(esc:ExtractedSectionContent)
WHERE esc.section_name IN ['FinancialStatements', 'FinancialStatementsandSupplementaryData']
RETURN c.ticker, r.formType, 
       substring(esc.content, 0, 2000) as financial_text,
       size(esc.content) as content_size
ORDER BY r.created DESC
LIMIT 10"""

data = await run_cypher_query.ainvoke(cypher_query)
data

[{'c.ticker': 'PNR',
  'r.formType': '10-Q',
  'financial_text': ' PART I FINANCIAL INFORMATION \n\nITEM 1. FINANCIAL STATEMENTS \n\nPentair plc and Subsidiaries \n\nCondensed Consolidated Statements of Operations and Comprehensive Income (Unaudited) \n\nThree months ended Six months ended In millions, except per-share data June 30, 2025 June 30, 2024 June 30, 2025 June 30, 2024 Net sales $ 1,123.1   $ 1,099.3   $ 2,133.5   $ 2,116.5   Cost of goods sold 666.5   661.4   1,273.6   1,288.5   Gross profit 456.6   437.9   859.9   828.0   Selling, general and administrative expenses 213.8   165.1   390.4   350.3   Research and development expenses 25.1   24.8   48.7   48.9   Operating income 217.7   248.0   420.8   428.8   Other expense \n\nLoss on sale of business \n\n26.3   —   26.3   —   Net interest expense 17.9   26.3   37.6   53.6   \n\nOther expense \n\n1.0   0.8   1.5   0.9   Income from continuing operations before income taxes 172.5   220.9   355.4   374.3   Provision for income t

In [7]:
cypher_query = """MATCH (r:Report)-[:HAS_XBRL]->(x:XBRLNode)<-[:REPORTS]-(f:Fact)-[:HAS_CONCEPT]->(c:Concept)
WHERE c.qname = 'us-gaap:RevenueFromContractWithCustomerExcludingAssessedTax' 
  AND f.is_numeric = '1'
WITH r, x, f, c LIMIT 5
MATCH (f)-[:HAS_PERIOD]->(p:Period)
MATCH (f)-[:HAS_UNIT]->(u:Unit)
MATCH (f)-[:IN_CONTEXT]->(ctx:Context)-[:FOR_COMPANY]->(comp:Company)
OPTIONAL MATCH (f)-[:FACT_MEMBER]->(m:Member)
RETURN comp.ticker, r.formType, f.value as revenue, c.label,
       p.period_type, p.start_date, p.end_date, u.name as unit,
       COLLECT(m.label) as dimensions"""

data = await run_cypher_query.ainvoke(cypher_query)
data

[{'comp.ticker': 'DOCN',
  'r.formType': '10-K',
  'revenue': '428,561,000',
  'c.label': 'Revenue from Contract with Customer, Excluding Assessed Tax',
  'p.period_type': 'duration',
  'p.start_date': '2021-01-01',
  'p.end_date': '2022-01-01',
  'unit': 'iso4217:USD',
  'dimensions': []},
 {'comp.ticker': 'DOCN',
  'r.formType': '10-K',
  'revenue': '576,322,000',
  'c.label': 'Revenue from Contract with Customer, Excluding Assessed Tax',
  'p.period_type': 'duration',
  'p.start_date': '2022-01-01',
  'p.end_date': '2023-01-01',
  'unit': 'iso4217:USD',
  'dimensions': []},
 {'comp.ticker': 'DOCN',
  'r.formType': '10-K',
  'revenue': '318,380,000',
  'c.label': 'Revenue from Contract with Customer, Excluding Assessed Tax',
  'p.period_type': 'duration',
  'p.start_date': '2020-01-01',
  'p.end_date': '2021-01-01',
  'unit': 'iso4217:USD',
  'dimensions': []},
 {'comp.ticker': 'CAG',
  'r.formType': '10-Q',
  'revenue': '5,712,200,000',
  'c.label': 'Revenue from Contract with Custo